In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/END

/content/drive/MyDrive/END


**Getting the dataset and saving in drive**

In [ ]:
# !wget https://s3-us-west-2.amazonaws.com/ai2-website/data/OpenBookQA-V1-Sep2018.zip

--2021-01-11 10:26:52--  https://s3-us-west-2.amazonaws.com/ai2-website/data/OpenBookQA-V1-Sep2018.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.221.72
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.221.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1446098 (1.4M) [application/zip]
Saving to: ‘OpenBookQA-V1-Sep2018.zip’

OpenBookQA-V1-Sep20 100%[===================>]   1.38M  1.44MB/s    in 1.0s    

2021-01-11 10:26:54 (1.44 MB/s) - ‘OpenBookQA-V1-Sep2018.zip’ saved [1446098/1446098]



In [ ]:
# !unzip /content/drive/MyDrive/END/OpenBookQA-V1-Sep2018.zip

Archive:  /content/drive/MyDrive/END/OpenBookQA-V1-Sep2018.zip
   creating: OpenBookQA-V1-Sep2018/
   creating: OpenBookQA-V1-Sep2018/Data/
   creating: OpenBookQA-V1-Sep2018/Data/Additional/
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/test_complete.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/train_complete.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/crowdsourced-facts.txt  
  inflating: OpenBookQA-V1-Sep2018/Data/Additional/dev_complete.jsonl  
   creating: OpenBookQA-V1-Sep2018/Data/Main/
  inflating: OpenBookQA-V1-Sep2018/Data/Main/train.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/test.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/train.tsv  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/dev.tsv  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/dev.jsonl  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/openbook.txt  
  inflating: OpenBookQA-V1-Sep2018/Data/Main/test.tsv  


In [ ]:
# !rm -r /content/drive/MyDrive/END/OpenBookQA-V1-Sep2018.zip

**Imports**

In [3]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from tqdm.notebook import tqdm
import torchtext
import spacy
from spacy.lang.zh import Chinese
from torchtext.data import Field
import torch
import torch.nn as nn
from torchtext.data import BucketIterator
import torch.optim as optim
import time
import random
import math
from torchtext import data as data

In [4]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


**Let's create a function creat_dataframe() to convert the json file to a dataframe for ease of use**

In [6]:
def retrieve_answer(row):
  correct_answer = row['answerKey']
  choices = row['answer_choice']
  return [x['text'] for x in choices if x['label'] == correct_answer][0]

In [7]:
def create_dataframe(data):
  data['question_stem'] = data['question'].apply(lambda x:x['stem'])
  data['answer_choice'] = data['question'].apply(lambda x:x['choices'])
  data.drop('question', axis=1, inplace = True)
  data['answer'] = data.apply(lambda row:retrieve_answer(row), axis = 1)
  data.rename(columns = {'question_stem':'question'}, inplace = True)
  cols_to_return = ['question', 'answer']
  return data[cols_to_return]

In [8]:
train_df = pd.read_json('/content/drive/MyDrive/END/OpenBookQA-V1-Sep2018/Data/Main/train.jsonl', lines = True)
valid_df = pd.read_json('/content/drive/MyDrive/END/OpenBookQA-V1-Sep2018/Data/Main/dev.jsonl', lines = True)
test_df = pd.read_json('/content/drive/MyDrive/END/OpenBookQA-V1-Sep2018/Data/Main/test.jsonl', lines = True)

In [9]:
create_dataframe(train_df.copy())

,question,answer
0,The sun is responsible for,"plants sprouting, blooming and wilting"
1,When standing miles away from Mount Rushmore,the mountains seem smaller than in photographs
2,When food is reduced in the stomach,nutrients are being deconstructed
3,Stars are,great balls of gas burning billions of miles away
4,You can make a telescope with a,mailing tube
...,...,...
4952,A bulldozer alters the area of,skyscrapers
4953,An organism that can survive without the help ...,Brewer's yeast
4954,The nimbleness of this animal is a key adaptio...,the antelope
4955,Birds will have different kinds of beaks depen...,organisms they hunt


In [10]:
train_df = create_dataframe(train_df.copy())
valid_df = create_dataframe(valid_df.copy())
test_df = create_dataframe(test_df.copy())

In [11]:
train_df.shape, valid_df.shape, test_df.shape

((4957, 2), (500, 2), (500, 2))

In [12]:
print("Train Data:")
print(train_df.head())

print("\nValid Data:")
print(valid_df.head())

print("\nTest Data:")
print(test_df.head())

Train Data:
                                       question                                             answer
0                    The sun is responsible for             plants sprouting, blooming and wilting
1  When standing miles away from Mount Rushmore     the mountains seem smaller than in photographs
2           When food is reduced in the stomach                  nutrients are being deconstructed
3                                     Stars are  great balls of gas burning billions of miles away
4               You can make a telescope with a                                       mailing tube

Valid Data:
                                            question                          answer
0  Frilled sharks and angler fish live far beneat...                Deep sea animals
1  Gas can fill any container it is given, and li...              uses what it needs
2  When birds migrate south for the winter, they ...  they are genetically called to
3  If a person walks in the opposite dire

**Load the english language model from spacy and create tokenize function**

In [13]:
spacy_en = spacy.load('en')
def tokenize(text):
  return [token.text for token in spacy_en.tokenizer(text)]

**Since we wan't the Question and Answer to have the same vocabulary, we will create a single field object called text field for both question and answer**

In [14]:
text_field = Field(
    sequential=True,
    tokenize=tokenize, 
    lower=True,
    init_token='<sos>', eos_token='<eos>'
)

fields = [('question', text_field),('answer', text_field)]

In [15]:
example_train = [data.Example.fromlist([train_df.question[i],train_df.answer[i]], fields) for i in range(train_df.shape[0])] 
example_valid = [data.Example.fromlist([valid_df.question[i],valid_df.answer[i]], fields) for i in range(valid_df.shape[0])] 
example_test = [data.Example.fromlist([test_df.question[i],test_df.answer[i]], fields) for i in range(test_df.shape[0])]

In [16]:
# Creating datasets
train_data = data.Dataset(example_train, fields)
valid_data = data.Dataset(example_valid, fields)
test_data = data.Dataset(example_test, fields)

**Let's look at few examples from the train, valid and test datasets**

In [17]:
def get_example(data, example_number):
    print(f"Question : \n\n{' '.join(data.examples[example_number].question)}")
    print(f"\nAnswer : \n\n{' '.join(data.examples[example_number].answer)}")

In [18]:
get_example(train_data, 0)

Question : 

the sun is responsible for

Answer : 

plants sprouting , blooming and wilting


In [19]:
get_example(valid_data, 20)

Question : 

to have a positive impact of the environment

Answer : 

salvage plastic bottles instead of throwing them away


**Build Vocab from the training set**

In [20]:
text_field.build_vocab(train_data, min_freq =2)

In [21]:
len(text_field.vocab)

3934

**Both Question and Answers are sharing the same vocab here. We can look at an example to see that. Focus on 'the' token in both examples below**

In [22]:
for token in train_data.examples[0].question:
  print(f"Token_Text : {token} >>>>> Token_Integer: {text_field.vocab.stoi[token]}")

Token_Text : the >>>>> Token_Integer: 5
Token_Text : sun >>>>> Token_Integer: 77
Token_Text : is >>>>> Token_Integer: 7
Token_Text : responsible >>>>> Token_Integer: 880
Token_Text : for >>>>> Token_Integer: 24


In [23]:
for token in train_data.examples[1].answer:
  print(f"Token_Text : {token} >>>>> Token_Integer: {text_field.vocab.stoi[token]}")

Token_Text : the >>>>> Token_Integer: 5
Token_Text : mountains >>>>> Token_Integer: 516
Token_Text : seem >>>>> Token_Integer: 2038
Token_Text : smaller >>>>> Token_Integer: 885
Token_Text : than >>>>> Token_Integer: 95
Token_Text : in >>>>> Token_Integer: 11
Token_Text : photographs >>>>> Token_Integer: 0


**Create iterators for train, valid and test data**

In [24]:
#Assign device to use
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

#Define Batch Size
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, sort = False,
    device = device)

**Let's look at a batch from the training iterator**

In [25]:
one_batch = next(iter(train_iterator))

In [26]:
print(f"Shape of a batch question: {one_batch.question.shape}\n")
print(f"Batch Question : {one_batch.question}\n")
print(f"Shape of a batch answer: {one_batch.answer.shape}\n")
print(f"Batch Answer : {one_batch.answer}\n")

Shape of a batch question: torch.Size([36, 128])

Batch Question : tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [ 217,  117,   11,  ...,   18,   64,  159],
        [ 970,   95,  121,  ...,  275,  246, 1233],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

Shape of a batch answer: torch.Size([14, 128])

Batch Answer : tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   0,    0, 2631,  ..., 2741, 3669,   61],
        [1759,    3,    3,  ...,    3,    3,    5],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')



**Now Let's build the Seq2Seq Model**

**ENCODER**


In [27]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

**DECODER**

In [28]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden


**Seq2Seq**

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [30]:
INPUT_DIM = len(text_field.vocab)
OUTPUT_DIM = len(text_field.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
ENC_DROPOUT = 0.6
DEC_DROPOUT = 0.6

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [31]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3934, 128)
    (rnn): GRU(128, 256)
    (dropout): Dropout(p=0.6, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3934, 128)
    (rnn): GRU(384, 256)
    (fc_out): Linear(in_features=640, out_features=3934, bias=True)
    (dropout): Dropout(p=0.6, inplace=False)
  )
)

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,318,302 trainable parameters


In [33]:
optimizer = optim.AdamW(model.parameters())
PAD_IDX = text_field.vocab.stoi[text_field.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

**Training Loop and Function**

In [34]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question
        trg = batch.answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

**Evaluation Function**

In [35]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            trg = batch.answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [37]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'OpenBookQA_model1.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 6.409 | Train PPL: 607.028
	 Val. Loss: 5.458 |  Val. PPL: 234.534
Epoch: 02 | Time: 0m 1s
	Train Loss: 5.368 | Train PPL: 214.410
	 Val. Loss: 5.418 |  Val. PPL: 225.381
Epoch: 03 | Time: 0m 1s
	Train Loss: 5.235 | Train PPL: 187.688
	 Val. Loss: 5.393 |  Val. PPL: 219.812
Epoch: 04 | Time: 0m 1s
	Train Loss: 5.165 | Train PPL: 174.993
	 Val. Loss: 5.427 |  Val. PPL: 227.487
Epoch: 05 | Time: 0m 1s
	Train Loss: 5.096 | Train PPL: 163.371
	 Val. Loss: 5.438 |  Val. PPL: 229.990
Epoch: 06 | Time: 0m 1s
	Train Loss: 5.026 | Train PPL: 152.274
	 Val. Loss: 5.443 |  Val. PPL: 231.126
Epoch: 07 | Time: 0m 1s
	Train Loss: 4.974 | Train PPL: 144.623
	 Val. Loss: 5.455 |  Val. PPL: 234.025
Epoch: 08 | Time: 0m 1s
	Train Loss: 4.947 | Train PPL: 140.732
	 Val. Loss: 5.482 |  Val. PPL: 240.229
Epoch: 09 | Time: 0m 1s
	Train Loss: 4.918 | Train PPL: 136.763
	 Val. Loss: 5.487 |  Val. PPL: 241.465
Epoch: 10 | Time: 0m 1s
	Train Loss: 4.867 | Train PPL: 129.875


In [38]:
model.load_state_dict(torch.load('OpenBookQA_model1.pt'))

<All keys matched successfully>

In [39]:
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.219 | Test PPL: 184.824 |


In [40]:
def show_predictions(question, answer, prediction, example_id):
  model_prediction = " ".join([text_field.vocab.itos[x] for x in torch.argmax(prediction[:, example_id, :], axis = 1).cpu()])

  print("Question:\n")
  print(" ".join([text_field.vocab.itos[x] for x in question[:, example_id].cpu()]))
    

  print("\nActual Answer: \n")
  print(" ".join([text_field.vocab.itos[x] for x in answer[:, example_id].cpu()]))

  print("\nPredicted Answer: \n")
  print(model_prediction)

In [41]:
example_number = 0
test_iterator_list = list(test_iterator)
batch = test_iterator_list[example_number]
question = batch.question
answer = batch.answer
prediction = model(question, answer, 0)

In [42]:
show_predictions(question, answer, prediction, example_number)

Question:

<sos> a person wants to start <unk> money so that they can <unk> a nice <unk> at the end of the year . after looking over their <unk> and <unk> , they decide the best way to save money is to <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

Actual Answer: 

<sos> <unk> eating lunch out <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

Predicted Answer: 

<unk> <unk> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
